In [1]:
import pyspark 

In [2]:
spark  = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
ratings = spark.read.json('data/ratings.json')

In [4]:
ratings.persist()

DataFrame[movie_id: bigint, rating: bigint, timestamp: double, user_id: bigint]

In [5]:
ratings.show(5)

+--------+------+------------+-------+
|movie_id|rating|   timestamp|user_id|
+--------+------+------------+-------+
|     858|     4|9.56678732E8|   6040|
|    2384|     4|9.56678754E8|   6040|
|     593|     5|9.56678754E8|   6040|
|    1961|     4|9.56678777E8|   6040|
|    1419|     3|9.56678856E8|   6040|
+--------+------+------------+-------+
only showing top 5 rows



In [6]:
import pandas as pd 
movies = pd.read_csv('data/movies.dat', sep='::', engine='python', header=None)
len(movies)

3883

In [7]:
users = pd.read_csv('data/users.dat', sep='::', engine='python', header=None)
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
users = users.rename({0:'user_id', 
              1:'gender', 
              2:'min_age', 
              3:'occupation', 
              4:'zipcode'}, 
             axis=1)
users.min_age.value_counts()

25    2096
35    1193
18    1103
45     550
50     496
56     380
1      222
Name: min_age, dtype: int64

In [9]:
requests = spark.read.json('data/requests.json')

In [10]:
requests.persist()

DataFrame[movie_id: bigint, rating: double, timestamp: double, user_id: bigint]

In [11]:
requests.count()

280260

In [12]:
ratings_df = ratings.toPandas()

In [13]:
ratings_df.head()

,movie_id,rating,timestamp,user_id
0,858,4,956678732.0,6040
1,2384,4,956678754.0,6040
2,593,5,956678754.0,6040
3,1961,4,956678777.0,6040
4,1419,3,956678856.0,6040


In [14]:
from pyspark.ml.recommendation import ALSModel, ALS

In [15]:
als = ALS(
    rank=11,
    userCol='user_id',
    itemCol='movie_id',
    ratingCol='rating'
)

In [16]:
als_model = als.fit(ratings)

In [17]:
preds = als_model.transform(ratings)

In [18]:
request_preds = als_model.transform(requests)

In [19]:
nan_df = request_preds.toPandas()

In [20]:
nan_df.head()

,movie_id,rating,timestamp,user_id,prediction
0,148,NaN,977959026.0,53,NaN
1,148,NaN,976559602.0,4169,3.068630
2,148,NaN,989024856.0,5333,2.389346
3,148,NaN,977005381.0,4387,2.161703
4,148,NaN,966907208.0,3539,2.734761


In [21]:
import numpy as np
nan_df = nan_df[nan_df['prediction'].isnull()]

In [22]:
nan_df.head()

,movie_id,rating,timestamp,user_id,prediction
0,148,NaN,9.779590e+08,53,NaN
6,148,NaN,9.768416e+08,216,NaN
7,148,NaN,9.761912e+08,482,NaN
9,148,NaN,1.026978e+09,424,NaN
14,463,NaN,9.782428e+08,26,NaN


In [23]:
len(nan_df)

95628

In [24]:
meta_df = pd.read_csv('data/movies_metadata.csv')

/anaconda3/envs/learn-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
meta_df[meta_df.id.str.contains('-')==True]
#drop things that got shifted 

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
bad_ids = ['1997-08-20', '2012-09-29', '2014-01-01'] 

In [27]:
meta_df = meta_df[~meta_df['id'].isin(bad_ids)]
meta_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [28]:
meta_df['id'] = meta_df['id'].astype(int)

In [29]:
all_training_data_df = ratings_df.merge(meta_df, how='left', left_on='movie_id', right_on='id')

In [30]:
all_training_data_df = all_training_data_df.merge(users, how='left', left_on='user_id', right_on='user_id')

In [31]:
all_training_data_df.head().T

,0,1,2,3,4
movie_id,858,2384,593,1961,1419
rating,4,4,5,4,3
timestamp,9.56679e+08,9.56679e+08,9.56679e+08,9.56679e+08,9.56679e+08
user_id,6040,6040,6040,6040,6040
adult,False,NaN,False,False,False
belongs_to_collection,NaN,NaN,NaN,NaN,NaN
budget,21000000,NaN,0,1500000,0
genres,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...","[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
homepage,NaN,NaN,NaN,http://www.dhentertainment.com/projects/1.my-n...,http://www.vollidiot-derfilm.de/
id,858,NaN,593,1961,1419


In [32]:
all_data_df = nan_df.merge(meta_df, how='left', left_on='movie_id', right_on='id')

In [33]:
all_data_df.head()

,movie_id,rating,timestamp,user_id,prediction,adult,belongs_to_collection,budget,genres,homepage,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,148,NaN,9.779590e+08,53,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,2005-12-15,0.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,The Secret Life of Words,False,6.8,52.0
1,148,NaN,9.768416e+08,216,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,2005-12-15,0.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,The Secret Life of Words,False,6.8,52.0
2,148,NaN,9.761912e+08,482,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,2005-12-15,0.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,The Secret Life of Words,False,6.8,52.0
3,148,NaN,1.026978e+09,424,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,2005-12-15,0.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,The Secret Life of Words,False,6.8,52.0
4,463,NaN,9.782428e+08,26,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
all_data_df = nan_df.merge(meta_df, how='left', left_on='movie_id', right_on='id')

In [35]:
all_data_df = all_data_df.merge(users, how='left', left_on='user_id', right_on='user_id')
all_data_df.head(2)

,movie_id,rating,timestamp,user_id,prediction,adult,belongs_to_collection,budget,genres,homepage,...,status,tagline,title,video,vote_average,vote_count,gender,min_age,occupation,zipcode
0,148,NaN,977959026.0,53,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,Released,NaN,The Secret Life of Words,False,6.8,52.0,M,25,0,96931
1,148,NaN,976841639.0,216,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,Released,NaN,The Secret Life of Words,False,6.8,52.0,M,45,13,52761


In [36]:
all_data_df.head(1)

,movie_id,rating,timestamp,user_id,prediction,adult,belongs_to_collection,budget,genres,homepage,...,status,tagline,title,video,vote_average,vote_count,gender,min_age,occupation,zipcode
0,148,NaN,977959026.0,53,NaN,False,NaN,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,...,Released,NaN,The Secret Life of Words,False,6.8,52.0,M,25,0,96931


In [37]:
X = all_data_df.filter(['zipcode','occupation','min_age','gender','vote_count', 'vote_average', 
                'runtime', 'revenue', 'release_date', 'popularity', 
               'budget', 'adult', 'user_id', 'movie_id'], axis=1)

In [38]:
y = all_data_df.filter(['prediction'], axis=1)

In [39]:
from keras.models import Sequential
from keras.layers import Dense 
from keras.optimizers import Adam 

Using TensorFlow backend.


In [40]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [41]:
gender_dict = {'M':0, 'F':1}

In [42]:
X['gender'] = X['gender'].replace(gender_dict)

In [43]:
adult_dict = {'True':True, 'False':False}

In [44]:
X['adult'] = X['adult'].replace(adult_dict)
type(X['adult'][0])

bool

In [45]:
X['budget'] = X['budget'].astype(float)

In [46]:
X.drop('zipcode', axis=1, inplace=True)

In [47]:
X['release_date'] = pd.DatetimeIndex(X['release_date']).astype(np.int64)

In [48]:
X['popularity'].astype(np.float64, copy=False)
type(X['popularity'][0])

float

In [49]:
#one-hot encode occupation
enc_cols = X['occupation'].values.reshape(-1, 1)
encoder = OneHotEncoder().fit(enc_cols)
encoder.get_feature_names(['occupation'])

ohe = pd.DataFrame(encoder.transform(enc_cols).toarray(),
                   columns=encoder.get_feature_names(['occupation']))

X = pd.concat([X.drop(['occupation'], axis=1), ohe], axis=1, )

/anaconda3/envs/learn-env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [50]:
X.head().T

,0,1,2,3,4
min_age,25,45,25,25,25
gender,0,0,0,0,0
vote_count,52,52,52,52,NaN
vote_average,6.8,6.8,6.8,6.8,NaN
runtime,112,112,112,112,NaN
revenue,0,0,0,0,NaN
release_date,1134604800000000000,1134604800000000000,1134604800000000000,1134604800000000000,-9223372036854775808
popularity,12.7756,12.7756,12.7756,12.7756,NaN
budget,5e+06,5e+06,5e+06,5e+06,NaN
adult,False,False,False,False,NaN


In [51]:
X.drop(['occupation_0.0'], axis=1, inplace=True)

In [52]:
X.rename({'occupation_1.0':'academic_educator', 
          'occupation_2.0':'artist',
         'occupation_3.0':'clerical_admin',
         'occupation_4.0':'coll_grad_student',
         'occupation_5.0':'cust_service',
         'occupation_6.0':'doctor',
         'occupation_7.0':'exec',
         'occupation_8.0':'farmer',
         'occupation_9.0':'homemaker',
         'occupation_10.0':'young_student',
         'occupation_11.0':'lawyer',
         'occupation_12.0':'programmer',
         'occupation_13.0':'retired',
         'occupation_14.0':'sales_mkting',
          'occupation_15.0':'scientist',
         'occupation_16.0':'self_employed',
         'occupation_17.0':'tech_eng',
         'occupation_18.0':'tradesman',
          'occupation_19.0':'unemployed',
          'occupation_20.0':'writer',}, axis=1, inplace=True)


In [53]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95628 entries, 0 to 95627
Data columns (total 32 columns):
min_age              95628 non-null int64
gender               95628 non-null int64
vote_count           48000 non-null float64
vote_average         48000 non-null float64
runtime              48000 non-null float64
revenue              48000 non-null float64
release_date         95628 non-null int64
popularity           48000 non-null object
budget               48000 non-null float64
adult                48000 non-null object
user_id              95628 non-null int64
movie_id             95628 non-null int64
academic_educator    95628 non-null float64
artist               95628 non-null float64
clerical_admin       95628 non-null float64
coll_grad_student    95628 non-null float64
cust_service         95628 non-null float64
doctor               95628 non-null float64
exec                 95628 non-null float64
farmer               95628 non-null float64
homemaker            95

In [54]:
#X_3 will have more rows to train the model but we'll only take the predictions of the ones we still need
X_3 = X.drop(['vote_count', 'vote_average', 'runtime', 
                         'revenue', 'popularity', 'budget', 'adult'],axis=1)

In [55]:
X_2 = X.dropna()

In [56]:
X_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48000 entries, 0 to 95491
Data columns (total 32 columns):
min_age              48000 non-null int64
gender               48000 non-null int64
vote_count           48000 non-null float64
vote_average         48000 non-null float64
runtime              48000 non-null float64
revenue              48000 non-null float64
release_date         48000 non-null int64
popularity           48000 non-null object
budget               48000 non-null float64
adult                48000 non-null object
user_id              48000 non-null int64
movie_id             48000 non-null int64
academic_educator    48000 non-null float64
artist               48000 non-null float64
clerical_admin       48000 non-null float64
coll_grad_student    48000 non-null float64
cust_service         48000 non-null float64
doctor               48000 non-null float64
exec                 48000 non-null float64
farmer               48000 non-null float64
homemaker            48

In [57]:
X_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95628 entries, 0 to 95627
Data columns (total 25 columns):
min_age              95628 non-null int64
gender               95628 non-null int64
release_date         95628 non-null int64
user_id              95628 non-null int64
movie_id             95628 non-null int64
academic_educator    95628 non-null float64
artist               95628 non-null float64
clerical_admin       95628 non-null float64
coll_grad_student    95628 non-null float64
cust_service         95628 non-null float64
doctor               95628 non-null float64
exec                 95628 non-null float64
farmer               95628 non-null float64
homemaker            95628 non-null float64
young_student        95628 non-null float64
lawyer               95628 non-null float64
programmer           95628 non-null float64
retired              95628 non-null float64
sales_mkting         95628 non-null float64
scientist            95628 non-null float64
self_employed        

In [58]:
print(f'There are {95628-48000} predictions for our third model to make.')

There are 47628 predictions for our third model to make.


In [59]:
all_training_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 719949 entries, 0 to 719948
Data columns (total 32 columns):
movie_id                 719949 non-null int64
rating                   719949 non-null int64
timestamp                719949 non-null float64
user_id                  719949 non-null int64
adult                    361823 non-null object
belongs_to_collection    88134 non-null object
budget                   361823 non-null object
genres                   361823 non-null object
homepage                 90115 non-null object
id                       361823 non-null float64
imdb_id                  361823 non-null object
original_language        361823 non-null object
original_title           361823 non-null object
overview                 360854 non-null object
popularity               361823 non-null object
poster_path              361823 non-null object
production_companies     361823 non-null object
production_countries     361823 non-null object
release_date             361

In [60]:
X_train = all_training_data_df.filter(['zipcode','occupation','min_age','gender','vote_count', 'vote_average', 
                'runtime', 'revenue', 'release_date', 'popularity', 
               'budget', 'adult', 'user_id', 'movie_id', 'rating'], axis=1)

In [61]:
X_train['gender'] = X_train['gender'].replace(gender_dict)

In [62]:
X_train['adult'] = X_train['adult'].replace(adult_dict)
type(X_train['adult'][0])

bool

In [63]:
X_train['budget'] = X_train['budget'].astype(float)

In [64]:
X_train.drop('zipcode', axis=1, inplace=True)

In [65]:
X_train['release_date'] = pd.DatetimeIndex(X_train['release_date']).astype(np.int64)

In [66]:
X_train['popularity'].astype(np.float64, copy=False)
type(X_train['popularity'][0])

float

In [67]:
#one-hot encode occupation
enc_cols = X_train['occupation'].values.reshape(-1, 1)
encoder = OneHotEncoder().fit(enc_cols)
encoder.get_feature_names(['occupation'])

ohe = pd.DataFrame(encoder.transform(enc_cols).toarray(),
                   columns=encoder.get_feature_names(['occupation']))

X_train = pd.concat([X_train.drop(['occupation'], axis=1), ohe], axis=1, )

/anaconda3/envs/learn-env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [68]:
X_train.head().T

,0,1,2,3,4
min_age,25,25,25,25,25
gender,0,0,0,0,0
vote_count,630,NaN,364,107,25
vote_average,6.5,NaN,7.7,5.9,4.3
runtime,105,NaN,167,86,102
revenue,2.278e+08,NaN,0,173066,0
release_date,740880000000000000,-9223372036854775808,69897600000000000,1191196800000000000,1176249600000000000
popularity,10.2349,NaN,11.0598,8.70879,1.99742
budget,2.1e+07,NaN,0,1.5e+06,0
adult,False,NaN,False,False,False


In [69]:
X_train.drop(['occupation_0.0'], axis=1, inplace=True)

In [70]:
X_train.rename({'occupation_1.0':'academic_educator', 
          'occupation_2.0':'artist',
         'occupation_3.0':'clerical_admin',
         'occupation_4.0':'coll_grad_student',
         'occupation_5.0':'cust_service',
         'occupation_6.0':'doctor',
         'occupation_7.0':'exec',
         'occupation_8.0':'farmer',
         'occupation_9.0':'homemaker',
         'occupation_10.0':'young_student',
         'occupation_11.0':'lawyer',
         'occupation_12.0':'programmer',
         'occupation_13.0':'retired',
         'occupation_14.0':'sales_mkting',
          'occupation_15.0':'scientist',
         'occupation_16.0':'self_employed',
         'occupation_17.0':'tech_eng',
         'occupation_18.0':'tradesman',
          'occupation_19.0':'unemployed',
          'occupation_20.0':'writer',}, axis=1, inplace=True)


In [71]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 719949 entries, 0 to 719948
Data columns (total 33 columns):
min_age              719949 non-null int64
gender               719949 non-null int64
vote_count           361823 non-null float64
vote_average         361823 non-null float64
runtime              361823 non-null float64
revenue              361823 non-null float64
release_date         719949 non-null int64
popularity           361823 non-null object
budget               361823 non-null float64
adult                361823 non-null object
user_id              719949 non-null int64
movie_id             719949 non-null int64
rating               719949 non-null int64
academic_educator    719949 non-null float64
artist               719949 non-null float64
clerical_admin       719949 non-null float64
coll_grad_student    719949 non-null float64
cust_service         719949 non-null float64
doctor               719949 non-null float64
exec                 719949 non-null float64
far

In [72]:
#X_3 will have more rows to train the model but we'll only take the predictions of the ones we still need
X_3_train = X_train.drop(['vote_count', 'vote_average', 'runtime', 
                         'revenue', 'popularity', 'budget', 'adult', 'rating'],axis=1)
y_3_train = X_train.filter(['rating'], axis=1)

In [73]:
X_2_train = X_train.dropna()
y_2_train = X_2_train.filter(['rating'], axis=1)
X_2_train = X_2_train.drop(['rating'], axis=1)

In [74]:
X_2_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361823 entries, 0 to 719948
Data columns (total 32 columns):
min_age              361823 non-null int64
gender               361823 non-null int64
vote_count           361823 non-null float64
vote_average         361823 non-null float64
runtime              361823 non-null float64
revenue              361823 non-null float64
release_date         361823 non-null int64
popularity           361823 non-null object
budget               361823 non-null float64
adult                361823 non-null object
user_id              361823 non-null int64
movie_id             361823 non-null int64
academic_educator    361823 non-null float64
artist               361823 non-null float64
clerical_admin       361823 non-null float64
coll_grad_student    361823 non-null float64
cust_service         361823 non-null float64
doctor               361823 non-null float64
exec                 361823 non-null float64
farmer               361823 non-null float64
h

In [75]:
X_3_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 719949 entries, 0 to 719948
Data columns (total 25 columns):
min_age              719949 non-null int64
gender               719949 non-null int64
release_date         719949 non-null int64
user_id              719949 non-null int64
movie_id             719949 non-null int64
academic_educator    719949 non-null float64
artist               719949 non-null float64
clerical_admin       719949 non-null float64
coll_grad_student    719949 non-null float64
cust_service         719949 non-null float64
doctor               719949 non-null float64
exec                 719949 non-null float64
farmer               719949 non-null float64
homemaker            719949 non-null float64
young_student        719949 non-null float64
lawyer               719949 non-null float64
programmer           719949 non-null float64
retired              719949 non-null float64
sales_mkting         719949 non-null float64
scientist            719949 non-null float64

In [76]:
ss = StandardScaler()
X_2_scaled = ss.fit_transform(X_2)
X_2_train_scaled = ss.fit_transform(X_2_train)
X_3_train_scaled = ss.fit_transform(X_3_train)
X_3_scaled = ss.fit_transform(X_3)

In [146]:
model = Sequential()
inputs = X_2_scaled.shape[1]
hiddens = inputs
model.add(Dense(hiddens, input_dim=inputs, activation='relu'))
model.add(Dense(6, activation='softmax'))
adam=Adam()

In [147]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [148]:
from keras.utils import to_categorical
y_cat = to_categorical(y_2_train)

In [ ]:
history = model.fit(X_2_train_scaled, y_cat, epochs=250)

Epoch 1/250
361823/361823 [==============================] - 8s 21us/step - loss: 1.4585 - acc: 0.3424
Epoch 2/250
361823/361823 [==============================] - 8s 22us/step - loss: 1.4442 - acc: 0.3472
Epoch 3/250
361823/361823 [==============================] - 8s 23us/step - loss: 1.4406 - acc: 0.3486
Epoch 4/250
361823/361823 [==============================] - 7s 19us/step - loss: 1.4374 - acc: 0.3484
Epoch 5/250
361823/361823 [==============================] - 7s 21us/step - loss: 1.4341 - acc: 0.3495
Epoch 6/250
361823/361823 [==============================] - 8s 21us/step - loss: 1.4324 - acc: 0.3504
Epoch 7/250
361823/361823 [==============================] - 8s 21us/step - loss: 1.4306 - acc: 0.3520
Epoch 8/250
361823/361823 [==============================] - 7s 19us/step - loss: 1.4294 - acc: 0.3526
Epoch 9/250
361823/361823 [==============================] - 7s 19us/step - loss: 1.4283 - acc: 0.3525
Epoch 10/250
361823/361823 [==============================] - 7s 19us/ste

In [ ]:
requests_df = request_preds.toPandas()

In [ ]:


for i, row in requests_df.iterrows():
    if str(row['prediction']) == 'nan':
        if row['user_id'] in X_2['user_id']:
            requests_df.loc[i, 'prediction'] = model.predict_classes(pd.DataFrame(X_2.loc[row['user_id'],:]).T)

In [ ]:
model = Sequential()
inputs = X_3_scaled.shape[1]
hiddens = inputs
model.add(Dense(hiddens, input_dim=inputs, activation='relu'))
model.add(Dense(6, activation='softmax'))
adam=Adam()

In [ ]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.utils import to_categorical
y_cat = to_categorical(y_3_train)

In [ ]:
history = model.fit(X_3_train_scaled, y_cat, epochs=250)

In [ ]:
for i, row in requests_df.iterrows():
    if str(row['prediction']) == 'nan':
        if row['user_id'] in X_3['user_id']:
            requests_df.loc[i, 'prediction'] = model.predict_classes(pd.DataFrame(X_3.loc[row['user_id'],:]).T)

In [ ]:
requests_df
requests_df.to_json()

In [103]:

# with open('final_json.json', 'wb') as f:
#     json.dumps(requests_df.to_json())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [106]:
requests_df


,movie_id,rating,timestamp,user_id,prediction
0,148,NaN,9.779590e+08,53,5.000000
1,148,NaN,9.765596e+08,4169,3.068630
2,148,NaN,9.890249e+08,5333,2.389346
3,148,NaN,9.770054e+08,4387,2.161703
4,148,NaN,9.669072e+08,3539,2.734761
5,148,NaN,9.762665e+08,840,2.262279
6,148,NaN,9.768416e+08,216,5.000000
7,148,NaN,9.761912e+08,482,5.000000
8,148,NaN,1.029284e+09,752,3.038933
9,148,NaN,1.026978e+09,424,5.000000


In [105]:
requests_df.to_json('final_json.json')

In [139]:
requests_df = request_preds.toPandas()

In [140]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr.fit(X_2_train, y_2_train)

/anaconda3/envs/learn-env/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [141]:
for i, row in requests_df.iterrows():
    if str(row['prediction']) == 'nan':
        if row['user_id'] in X_2['user_id']:
            requests_df.loc[i, 'prediction'] = lr.predict(pd.DataFrame(X_2.loc[row['user_id'],:]).T)

In [142]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr.fit(X_3_train, y_3_train)

/anaconda3/envs/learn-env/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [143]:
for i, row in requests_df.iterrows():
    if str(row['prediction']) == 'nan':
        if row['user_id'] in X_3['user_id']:
            requests_df.loc[i, 'prediction'] = lr.predict(pd.DataFrame(X_3.loc[row['user_id'],:]).T)

In [144]:
requests_df.to_json('final_json_logistic.json')

In [145]:
requests_df

,movie_id,rating,timestamp,user_id,prediction
0,148,NaN,9.779590e+08,53,4.000000
1,148,NaN,9.765596e+08,4169,3.068630
2,148,NaN,9.890249e+08,5333,2.389346
3,148,NaN,9.770054e+08,4387,2.161703
4,148,NaN,9.669072e+08,3539,2.734761
5,148,NaN,9.762665e+08,840,2.262279
6,148,NaN,9.768416e+08,216,1.000000
7,148,NaN,9.761912e+08,482,4.000000
8,148,NaN,1.029284e+09,752,3.038933
9,148,NaN,1.026978e+09,424,1.000000


In [ ]:
movies_avg = ratings_df.groupby('movie_id').mean()

In [ ]:
users_avg = ratings_df.groupby('user_id').mean()
users_avg.head()

In [ ]:
request_df.head()

In [ ]:
for i, row in request_df.iterrows():
    if str(row['prediction']) == 'nan':
        for user in users_avg.iterrows():
            if user.index == row['user_id']:
                for movie in movies_avg:
                    if movie['movie_id'] == row['movie_id']:
                        request_df.loc[i, 'prediction'] = (user['rating'] * movie['rating'])/2

In [ ]:
request_df.to_json('final_final_json.json')

In [ ]:
request_df